In [1]:
import numpy as np
import os
import cv2
import re
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
os.chdir('./cap_string_test/')


In [2]:
frame_dir = '11_27_test/'
framelist = os.listdir('.')
frame_out_dir = '../video_frames_out'
framelen = len(framelist)
framelist.sort()


In [3]:
try:
    os.mkdir(frame_out_dir)
except:
    pass

csv_dir = '../../data/001/gaze/'
csv_file = '001_gaze_capitalizeString_1412807.csv'


In [21]:
df = pd.read_csv(csv_dir + csv_file)
df.columns = ["pid", "function_name", "function_id", "system_timestamp", "device_timestamp", "valid_gaze_left",
              "valid_gaze_right", "gaze_left", "gaze_right", "valid_pd_left", "valid_pd_right", "gaze_left", "gaze_right"]

dflen = len(df)
df_valid = df[(df['valid_gaze_left'] == 1)]  # FIXME - column names
df_valid = df[(df['valid_gaze_right'] == 1)]
df_valid = df[(df['valid_pd_right'] == 1)]
df_valid = df[(df['valid_pd_left'] == 1)]


(8258, 13)


In [27]:
img_path = os.getcwd() + '/' + framelist[0]

img = cv2.imread(img_path)
dimensions = img.shape
# height, width, number of channels in image
height = img.shape[0]
width = img.shape[1]


height: 720
width: 1280


In [51]:
starttime = int(df_valid.iloc[0, 3])
print(starttime/(10**6))

endtime = int(df_valid.iloc[-1, 3])
print(endtime/(10**6))
#print(df_valid.shape)

duration = endtime-starttime
print(duration/(10**6))

3980.666945
4050.236108
69.569163


In [36]:
for file in framelist:
    digits = re.findall('\d+', file)
    

1170

In [ ]:
def closest(lst, K):
    return lst[min(range(len(lst)), key=lambda i: abs(lst[i]-K))]


time_series = df['AdjustedTime']  # FIXME - column names and deltas
valid_time_series = df_valid['AdjustedTime']  # FIXME - same as above
